In [1]:
#################################################################################
"""Load Configuration Details loaded from configuration File

"""
 #Who                 When           What
 #Victor Salesa       19/02/2019     Initial Version
 #Victor Salesa       18/03/2019     Disable Warnings for Yaml load
 #Victor Salesa       14/04/2019     Added __JAYDEBE__ parameters
 #Ana Perez           02/05/2019     Added __MASTER_DATA_INGESTION_BASE_PATH__ variable
 #Ana Perez           06/05/2019     Added __MASTER_DATA_ARCHIVE_BASE_PATH__ variable
 #Victor Salesa       09/05/2019     Added one extra available environment unit-test
#################################################################################

import yaml
yaml.warnings({'YAMLLoadWarning': False})
import os
import traceback
from pprint import pprint

__ADP_ENVIRONMENTS__ = ['dev','test','unit-test','uat','prod','preprod']
__ADP_ENVIRONMENTS_MAX_LENGTH__ = __builtin__.max([len(env) for env in __ADP_ENVIRONMENTS__ ])

#Read configuration file path from ADP_CONFIG environment variable
try:
   __CONFIG_FILE_PATH__ = os.environ["ADP_CONFIG"].strip()
except KeyError as exc:
  traceback.print_exc()
  raise KeyError("ADP_CONFIG environment variable with ADP Configuration file path not set:"+str(exc))

#Read,Parse and load configuration file data 
try:
  with open(__CONFIG_FILE_PATH__, 'r') as stream:
    try:
      __ADP_CONFIG_ALL__ = yaml.load(stream)
    except Exception as exc:
      traceback.print_exc()
      raise Exception("Could not parse configuration Error Should be YAML format: "+str(exc))
except IOError as exc:
  traceback.print_exc()
  raise IOError("Could not find or read configuration file:"+ __CONFIG_FILE_PATH__+" "+exc)

#Read current environment from ADP_ENVIRONMENT environment variable
try:
  __ADP_ENVIRONMENT__ = os.environ["ADP_ENVIRONMENT"].lower()[0:__ADP_ENVIRONMENTS_MAX_LENGTH__].strip()
  if __ADP_ENVIRONMENT__ not in __ADP_ENVIRONMENTS__:
    raise Exception("Environment not valid: '"+__ADP_ENVIRONMENT__+"' should be one of "+str(__ADP_ENVIRONMENTS__))
except Exception as exc:
  traceback.print_exc()
  raise Exception("ADP_ENVIRONMENT environment variable not set Please set value as one of "+str(__ADP_ENVIRONMENTS__),exc)

#Retrieve just current environment configuration from loaded config  
try:
  __ADP_CONFIG_CURRENT__ = __ADP_CONFIG_ALL__[__ADP_ENVIRONMENT__]
except KeyError as exc:
  traceback.print_exc()
  raise KeyError("Environment Value not loaded successfully: __ADP_ENVIRONMENT__: '"+str(__ADP_ENVIRONMENT__)+"' should be one of "+str(__ADP_ENVIRONMENTS__))
except TypeError as exc:
  traceback.print_exc()
  raise TypeError("Config data not loaded successfully: __ADP_CONFIG_ALL__: "+str(__ADP_CONFIG_ALL__)+" "+str(exc))
##############################################################################################################################################################  

#Load Default values for parameters in configuration file

try:
  #Load Base Paths Configuration 
  for key in __ADP_CONFIG_CURRENT__["base_paths"].keys():
    if __ADP_CONFIG_CURRENT__["base_paths"][key][-1:]!='/':
      __ADP_CONFIG_CURRENT__["base_paths"][key]=__ADP_CONFIG_CURRENT__["base_paths"][key]+'/'
    
  __TOBEPROCESSED_BASE_PATH__ = __ADP_CONFIG_CURRENT__["base_paths"]["tobeprocessed"]
  __LANDING_BASE_PATH__       = __ADP_CONFIG_CURRENT__["base_paths"]["landing"]
  __ERROR_BASE_PATH__         = __ADP_CONFIG_CURRENT__["base_paths"]["error"]
  __CANONICAL_BASE_PATH__     = __ADP_CONFIG_CURRENT__["base_paths"]["canonical"]
  __ERRORPROCESS_BASE_PATH__  = __ADP_CONFIG_CURRENT__["base_paths"]["errorprocess"]
  __ARCHIVE_BASE_PATH__       = __ADP_CONFIG_CURRENT__["base_paths"]["archive"]
  __QUALITY_BASE_PATH__       = __ADP_CONFIG_CURRENT__["base_paths"]["quality"]
  __INGESTION_BASE_PATH__     = __ADP_CONFIG_CURRENT__["base_paths"]["ingestion"]
  
  __MASTER_DATA_LANDING_BASE_PATH__   = __LANDING_BASE_PATH__   + 'masterdata/'
  __MASTER_DATA_TOBEPROCESSED_BASE_PATH__   = __TOBEPROCESSED_BASE_PATH__   + 'masterdata/'
  __MASTER_DATA_CANONICAL_BASE_PATH__ = __CANONICAL_BASE_PATH__ + 'masterdata/'
  __MASTER_DATA_INGESTION_BASE_PATH__ = __INGESTION_BASE_PATH__ + 'masterdata/'
  __MASTER_DATA_ARCHIVE_BASE_PATH__ = __ARCHIVE_BASE_PATH__ + 'masterdata/'
  __MASTERDATA_FTP_FOLDER__ = '/masterdata'
  
  #Blob default mount point root defined for all the blob mounts
  __BLOB_MOUNT_POINT_ROOT__ = __ADP_CONFIG_CURRENT__["blob"]["mount_point_root"]
  if __BLOB_MOUNT_POINT_ROOT__[-1:] != '/':
    __BLOB_MOUNT_POINT_ROOT__ = __BLOB_MOUNT_POINT_ROOT__ + '/'
  
  #Blob access information
  __BLOB_ACCOUNT_NAME__     = __ADP_CONFIG_CURRENT__["blob"]["credentials"]["account_name"]
  if __ADP_CONFIG_CURRENT__["blob"]["credentials_mode"]=='plain':
    __BLOB_ACCOUNT_KEY__      = __ADP_CONFIG_CURRENT__["blob"]["credentials"]["account_key"]
  elif __ADP_CONFIG_CURRENT__["blob"]["credentials_mode"]=='dbsecrets':
    password_secrets_scope = __ADP_CONFIG_CURRENT__["blob"]["credentials"]["dbsecrets_password_scope"]
    password_secrets_key =   __ADP_CONFIG_CURRENT__["blob"]["credentials"]["dbsecrets_password_key"]
    __BLOB_ACCOUNT_KEY__ = dbutils.secrets.get(key=password_secrets_key, scope=password_secrets_scope)
    
  #Database Access information
  __JDBC_CLASSNAME__ = __ADP_CONFIG_CURRENT__["database"]["jdbcClassname"]
  __JDBC_DATABASE__  = __ADP_CONFIG_CURRENT__["database"]["jdbcDatabase"]
  __JDBC_HOSTNAME__  = __ADP_CONFIG_CURRENT__["database"]["jdbcHostname"]
  __JDBC_PORT__      = __ADP_CONFIG_CURRENT__["database"]["jdbcPort"]
  __JDBC_USERNAME__  = __ADP_CONFIG_CURRENT__["database"]["credentials"]["jdbcUsername"]
  
  if __ADP_CONFIG_CURRENT__["database"]["credentials_mode"]=='plain':
    __JDBC_PASSWORD__ = __ADP_CONFIG_CURRENT__["database"]["credentials"]["jdbcPassword"]
  elif __ADP_CONFIG_CURRENT__["database"]["credentials_mode"]=='dbsecrets':
    password_secrets_scope = __ADP_CONFIG_CURRENT__["database"]["credentials"]["dbsecrets_password_scope"]
    password_secrets_key =   __ADP_CONFIG_CURRENT__["database"]["credentials"]["dbsecrets_password_key"]
    __JDBC_PASSWORD__ = dbutils.secrets.get(key=password_secrets_key, scope=password_secrets_scope)
    
  __JDBC_URL__ = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4}".format(__JDBC_HOSTNAME__, __JDBC_PORT__,__JDBC_DATABASE__, __JDBC_USERNAME__, __JDBC_PASSWORD__)
  
  __JAYDEBE_DB_ELEMENT__ = ";databaseName={}".format(__JDBC_DATABASE__)
  __JAYDEBE_URL__ = (
      "jdbc:sqlserver://{host}:{port}{db_elem}"
      ";user={user};password={password}".format(
          host=__JDBC_HOSTNAME__, port=__JDBC_PORT__, db_elem=__JAYDEBE_DB_ELEMENT__,
          user=__JDBC_USERNAME__, password=__JDBC_PASSWORD__)
  )
  
  #Spain Farmatic Ftp Access information
  
  __FARMATIC_FTP_HOST__ = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["host"]
  __FARMATIC_FTP_PORT__ = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["port"]
  __FARMATIC_FTP_ACCT__ = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["acct"]
  
  __FARMATIC_FTP_USER__   = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["credentials"]["user"]
  if __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["credentials_mode"]=='plain':
    __FARMATIC_FTP_PASSWD__ = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["credentials"]["password"]
  elif __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["credentials_mode"]=='dbsecrets':
    password_secrets_scope = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["credentials"]["dbsecrets_password_scope"]
    password_secrets_key = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["ftp"]["credentials"]["dbsecrets_password_key"]
    __FARMATIC_FTP_PASSWD__ = dbutils.secrets.get(key=password_secrets_key, scope=password_secrets_scope)
  
  if __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["zip"]["credentials_mode"]=='plain':
    __FARMATIC_ZIP_PASSWD__ = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["zip"]["credentials"]["password"]
  elif __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["zip"]["credentials_mode"]=='dbsecrets':
    password_secrets_scope = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["zip"]["credentials"]["dbsecrets_password_scope"]
    password_secrets_key = __ADP_CONFIG_CURRENT__["pms"]["farmatic"]["zip"]["credentials"]["dbsecrets_password_key"]
    __FARMATIC_ZIP_PASSWD__ = dbutils.secrets.get(key=password_secrets_key, scope=password_secrets_scope)
  
except KeyError as exc:
  traceback.print_exc()
  raise KeyError("Configuration Value not set in configuration file: "+__CONFIG_FILE_PATH__)